In [1]:
# Define model
def DefineModel(Model):
    Model.Select = pyo.Var(Model.Candidate, domain = pyo.Binary)
    Model.Allocation = pyo.Var(Model.Item, Model.Candidate, within = pyo.Binary, initialize = 0)

    def rule_LBWidth(Model, i):   # Width of allocated product must be at least width of each item it is allocated to
        return sum(Model.Allocation[i, c] * Model.CandidateWidth[c] for c in Model.Candidate) >= Model.Width[i]
    Model.MinWidth = pyo.Constraint(Model.Item, rule = rule_LBWidth)

    def rule_LBLength(Model, i):   # Length of allocated product must be at least length of each item it is allocated to
        return sum(Model.Allocation[i, c] * Model.CandidateLength[c] for c in Model.Candidate) >= Model.Length[i]
    Model.MinLength = pyo.Constraint(Model.Item, rule = rule_LBLength)
    
    def rule_count(Model):   # Select the specified number of products that we want to order
        return sum(Model.Select[c] for c in Model.Candidate) == Model.Orders
    Model.NumOrders = pyo.Constraint(rule = rule_count)

    def rule_only(Model, i, c):   # Allocate an item to a candidate only if that candidate is selected
        return Model.Allocation[i, c] <= Model.Select[c]
    Model.SelectedOnly = pyo.Constraint(Model.Item, Model.Candidate, rule = rule_only)
    
    def rule_once(Model, i):   # Each item is allocated to exactly one product
        return sum(Model.Allocation[i, c] for c in Model.Candidate) == 1
    Model.AllocateOnce = pyo.Constraint(Model.Item, rule = rule_once)

    def rule_Obj(Model):   # Minimize waste = Area of allocated product minus area of item, in total for all items
        return sum(sum(Model.Allocation[i, c] * Model.CandidateArea[c] * Model.Weight[i] for c in Model.Candidate) for i in Model.Item) \
               - sum(Model.Width[i] * Model.Length[i] * Model.Weight[i] for i in Model.Item)
    Model.Obj = pyo.Objective(rule = rule_Obj, sense = pyo.minimize)